In [90]:
import pandas as pd
import numpy as np
#import sklearn.preprocessing.LabelBinarizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
import itertools
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go



In [91]:
encoded_df = pd.read_csv('../data/processed/02_b_encoded.csv',index_col = 0, header = [0,1])



In [92]:
mask = ~ (encoded_df.columns.get_level_values(level = 0)=='DevType')
non_DevType = encoded_df.iloc[:,mask].values
np.isnan(non_DevType).sum()

0

In [93]:
encoded_df.head()


LanguageHaveWorkedWith                                                    \
                     APL Assembly Bash/Shell    C    C#  C++ COBOL Clojure   
0                    0.0      0.0       19.5  0.0   0.0  0.0   0.0     0.0   
1                    0.0      0.0        0.0  0.0   0.0  0.0   0.0     0.0   
2                    0.0      0.0        0.0  0.0   0.0  9.0   0.0     0.0   
3                    0.0      0.0        0.0  0.0  31.5  0.0   0.0     0.0   
4                    0.0      0.0       10.0  0.0   0.0  5.0   0.0     5.0   

                ...        DevType                             \
  Crystal Dart  ... Engineer, data Engineer, site reliability   
0     0.0  0.0  ...              1                          0   
1     0.0  0.0  ...              0                          0   
2     0.0  0.0  ...              0                          0   
3     0.0  0.0  ...              0                          0   
4     0.0  0.0  ...              0                          0   

                                                                               \
  Engineering manager Marketing or sales professional Other (please specify):   
0                   0                               0                       1   
1                   0                               0                       0   
2                   0                               0                       0   
3                   0                               0                       0   
4                   0                               0                       0   

                                                                          \
  Product manager Scientist Senior Executive (C-Suite, VP, etc.) Student   
0               0         0                                    0       0   
1               0         0                                    0       0   
2               0         0                                    0       0   
3               0         0                                    0       0   
4               0         0                                    0       0   

                        
  System administrator  
0                    0  
1                    0  
2                    0  
3                    0  
4                    0  

[5 rows x 150 columns]

In [94]:
job_freq = encoded_df.DevType.sum().sort_values(ascending=False).reset_index()
job_freq.columns = ['job','freq']
job_freq['work'] = 'DevType'
job_freq

,job,freq,work
0,"Developer, full-stack",30360,DevType
1,"Developer, back-end",26380,DevType
2,"Developer, front-end",16424,DevType
3,"Developer, desktop or enterprise applications",9481,DevType
4,"Developer, mobile",8419,DevType
5,DevOps specialist,6560,DevType
6,System administrator,5307,DevType
7,Database administrator,5056,DevType
8,Designer,3850,DevType
9,Data scientist or machine learning specialist,3778,DevType


In [95]:
fig = px.treemap(job_freq,path = ['work','job'],color = 'freq', values = 'freq',color_continuous_scale='Blues')
fig.update_layout(width = 1000, height = 600)
fig.show()
fig.write_html('../reports/figures/job_freq_treemap.html')

/home/ayah/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/home/ayah/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [96]:
# skills_freq = encoded_df.drop('DevType', axis=1).sum().reset_index()
skills_freq = encoded_df._drop_axis('DevType', axis=1,level = 0).sum().reset_index()
skills_freq.columns = ['group', 'skill', 'freq']


In [97]:
skills_freq.sort_values('freq',ascending=False)

,group,skill,freq
95,ToolsTechHaveWorkedWith,Git,1082499.5
122,NEWCollabToolsHaveWorkedWith,Visual Studio Code,841371.0
19,LanguageHaveWorkedWith,JavaScript,774407.0
93,ToolsTechHaveWorkedWith,Docker,698353.5
16,LanguageHaveWorkedWith,HTML/CSS,626356.0
...,...,...,...
119,NEWCollabToolsHaveWorkedWith,TextMate,12701.5
97,ToolsTechHaveWorkedWith,Pulumi,12640.0
8,LanguageHaveWorkedWith,Crystal,10710.5
6,LanguageHaveWorkedWith,COBOL,6979.0


In [98]:
fig = px.treemap(skills_freq,path = ['group','skill'],color = 'freq', values = 'freq',color_continuous_scale='Blues')
fig.update_layout(width = 1500, height = 750)
fig.show()
fig.write_html('../reports/figures/skills_freq.treemap.html')

/home/ayah/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/home/ayah/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [99]:
jobs   = encoded_df.DevType.columns.tolist()
skills = encoded_df._drop_axis('DevType', axis=1,level = 0).columns.droplevel(level = 0).tolist()

In [100]:
#
job_skill = {}
mask = ~ (encoded_df.columns.get_level_values(level = 0)=='DevType')
non_DevType = encoded_df.iloc[:,mask].values

# non_DevType.shape()

for job in jobs:
    job_skill[job] = ((encoded_df['DevType'][job].values.reshape(1,-1)) @ non_DevType )[0]
    
job_skill = pd.DataFrame(job_skill,index = skills)
# job_skill
# job_skill = encoded_df['DevType'].values.reshape(len(jobs),-1) @ non_DevType
# job_skill = pd.DataFrame(job_skill,columns = skills,index = jobs)
     

In [101]:
job_skill = job_skill.T
job_skill.head()

,APL,Assembly,Bash/Shell,C,C#,C++,COBOL,Clojure,Crystal,Dart,...,RStudio,Rider,RubyMine,Sublime Text,TextMate,Vim,Visual Studio,Visual Studio Code,Webstorm,Xcode
Academic researcher,972.0,4312.5,18134.5,14484.5,9480.5,17373.5,777.5,1716.0,663.0,2723.5,...,7465.0,1639.0,858.0,8900.5,1068.0,15675.0,11155.0,26763.0,3180.5,5007.5
Data or business analyst,1434.5,3308.5,23809.5,10572.0,27781.0,12231.0,1065.0,2580.0,1184.0,3693.0,...,10394.5,3272.5,1220.0,13759.5,946.5,19465.5,30235.5,50195.5,5016.0,7887.0
Data scientist or machine learning specialist,924.0,4249.0,29302.0,16468.5,15117.0,22106.0,833.0,3340.5,874.0,3368.5,...,16400.0,1557.0,1144.5,15921.5,1264.5,27481.5,18226.5,50262.5,3967.0,7248.5
Database administrator,1173.0,6413.5,46648.5,20348.0,53973.0,23545.0,1789.5,2947.5,2357.0,8240.5,...,4826.0,6670.5,2212.5,22667.0,1983.0,36924.5,54814.0,82781.5,10282.5,16362.0
Designer,1159.0,5644.5,25714.5,16591.5,35476.5,20225.0,1452.5,1934.0,1280.0,6892.5,...,3130.0,3746.5,1216.0,17076.5,1336.5,18518.5,38791.5,60971.0,8097.0,14131.0


In [102]:
fig = ff.create_dendrogram(job_skill, orientation = 'left', labels = job_skill.index)
fig.show()
fig.write_html('../reports/figures/jobs_dendogram.html')

In [103]:
fig = go.Figure(data = go.Heatmap(z = job_skill, x =job_skill.columns , y = job_skill.index, colorscale= 'magma', ygap = 1))
fig.update_layout(width = 3000, height = 700)
fig.show()
fig.write_html('../reports/figures/skills_heatmap.html')

In [104]:
ss = StandardScaler()
std_job_skill = ss.fit_transform(job_skill)

In [105]:
fig = go.Figure(data = go.Heatmap(z = std_job_skill, x =job_skill.columns , y = job_skill.index, colorscale= 'magma', ygap = 1))
fig.update_layout(width = 3000, height = 700)
fig.show()
fig.write_html('../reports/figures/std_skills_heatmap.html')

In [106]:
non_backend_full_job_skill = job_skill.drop(["Developer, full-stack", "Developer, back-end"])

In [107]:
ss = StandardScaler()
std_job_skill = ss.fit_transform(non_backend_full_job_skill)

In [108]:
fig = go.Figure(data = go.Heatmap(z = std_job_skill, x =non_backend_full_job_skill.columns , y = non_backend_full_job_skill.index, colorscale= 'magma', ygap = 1))
fig.update_layout(width = 3000, height = 700)
fig.show()
fig.write_html('../reports/figures/std_skills_heatmap_dropped.html')

TODO
1. drop marketing
2. student/ other/ educator = not specified
3. merge scientist & researcher
4. subclustering on frontend & backend & fullstack
